<a href="https://colab.research.google.com/github/Jegovila/cursoVR/blob/main/Practica10%20Calibraci%C3%B3n%20de%20c%C3%A1mara%20pinhole/python/Pr%C3%A1ctica10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc "https://github.com/Jegovila/cursoVR/raw/main/Practica10%20Calibraci%C3%B3n%20de%20c%C3%A1mara%20pinhole/data/chessboard/cal1.zip"
!wget -nc "https://github.com/Jegovila/cursoVR/raw/main/Practica10%20Calibraci%C3%B3n%20de%20c%C3%A1mara%20pinhole/data/chessboard/cal2.zip"
!unzip -n cal1.zip
!unzip -n cal2.zip

In [13]:
import numpy as np
import cv2 as cv
import glob
import matplotlib.pyplot as plt
import yaml
from IPython import display
from time import sleep

In [14]:
filas = 6
cols = 8
square_size_mm = 28

scale_percent = .15 # reducción porcentaje

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objp = np.zeros((filas*cols,3), np.float32)
objp[:,:2] = np.mgrid[0:cols,0:filas].T.reshape(-1,2) * square_size_mm

images = glob.glob('./*.jpg')

In [ ]:
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret, corners = cv.findChessboardCorners(gray, (cols,filas), None)

    # Si encuentra el patrón
    if ret == True:
      plt.clf()
      objpoints.append(objp)
      corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
      imgpoints.append(corners2)

      # Dibuja las esquinas
      cv.drawChessboardCorners(img, (cols,filas), corners2, ret)
      plt.imshow(cv.cvtColor(cv.resize(img,(int(img.shape[1] * scale_percent),int(img.shape[0] * scale_percent))), cv.COLOR_BGR2RGB))

      display.display(plt.gcf())
      display.clear_output(wait=True)
      sleep(0.5)

In [16]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

img = cv.imread(images[0])
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h)) # alpha = 1 : zona sin distorsión

Rectificar

In [17]:
dst = cv.undistort(img, mtx, dist, None, newcameramtx)

Resultados imagen rectificada

In [ ]:
x, y, w, h = roi
cropped = dst[y:y+h, x:x+w]

fig, ax = plt.subplots(2, 2, figsize=(20,14))
fig.tight_layout()

plt.subplot(2,1,1)
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
plt.subplot(2,2,3)
plt.imshow(cv.cvtColor(dst, cv.COLOR_BGR2RGB))
plt.subplot(2,2,4)
plt.imshow(cv.cvtColor(cropped, cv.COLOR_BGR2RGB))

Resultados de calibración

In [ ]:
print("K = \n", mtx)
print("Parámetros de distorsión = \n", dist)
print("Vectores R: ", len(rvecs))
print("R: \n", rvecs)
print("Vectores t: ", len(tvecs))
print("t: \n", tvecs)

# Graficar patrones y cámara

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
for i in range(len(tvecs)):
    x = tvecs[i][0,0]
    y = tvecs[i][1,0]
    z = tvecs[i][2,0]
    ax.scatter(x, y, z, c='r')

    #plot reference frame
    xr = [[1], [0], [0]]
    yr = [[0], [1], [0]]
    zr = [[0], [0], [1]]
    R = cv.Rodrigues(rvecs[i])[0]

    P0 = np.transpose(R) @ xr
    P1 = np.transpose(R) @ yr
    P2 = np.transpose(R) @ zr

    ax.plot([x, x + float(P0[0]) * 50], [y, y + float(P0[1]) * 50], [z, z - float(P0[2]) * 50], color='r')
    ax.plot([x, x + float(P1[0]) * 50], [y, y + float(P1[1]) * 50], [z, z - float(P1[2]) * 50], color='g')
    ax.plot([x, x + float(P2[0]) * 50], [y, y + float(P2[1]) * 50], [z, z - float(P2[2]) * 50], color='b')

    X, Y = np.meshgrid(np.arange(x, x + P0[0] * 50), np.arange(y, y + P1[1] * 25))
    Z = -1 * (((-P2[0]*(X-x)-P2[1]*(Y-y))/P2[2])-z)
    #plt.contour(X, Y, Z, colors='black');
    ax.plot_surface(X, Y, Z, alpha=0.5)

# Graficar cámara
s = 20
ax.plot([-s, -s],[-s,-s],[-s, s],color='b')
ax.plot([s, s],[-s,-s],[-s, s],color='b')
ax.plot([-s, -s],[s,s],[-s, s],color='b')
ax.plot([s, s],[s,s],[-s, s],color='b')

ax.plot([-s, s],[-s,-s],[-s, -s],color='b')
ax.plot([s, s],[-s,s],[-s, -s],color='b')
ax.plot([-s, s],[s,s],[-s, -s],color='b')
ax.plot([-s, -s],[-s,s],[-s, -s],color='b')

ax.plot([-s, s],[-s,-s],[s, s],color='b')
ax.plot([s, s],[-s,s],[s, s],color='b')
ax.plot([-s, s],[s,s],[s, s],color='b')
ax.plot([-s, -s],[-s,s],[s, s],color='b')

ax.plot([-s, -1.5*s],[-s,-1.5*s],[s, 1.5*s],color='b')
ax.plot([s, 1.5*s],[-s,-1.5*s],[s, 1.5*s],color='b')
ax.plot([s, 1.5*s],[s,1.5*s],[s, 1.5*s],color='b')
ax.plot([-s, -1.5*s],[s,1.5*s],[s, 1.5*s],color='b')

ax.plot([-1.5*s, 1.5*s],[-1.5*s,-1.5*s],[1.5*s, 1.5*s],color='b')
ax.plot([1.5*s,1.5*s],[-1.5*s,1.5*s],[1.5*s, 1.5*s],color='b')
ax.plot([1.5*s, -1.5*s],[1.5*s,1.5*s],[1.5*s, 1.5*s],color='b')
ax.plot([-1.5*s,-1.5*s],[-1.5*s,1.5*s],[1.5*s, 1.5*s],color='b')

ax.scatter(0, 0, 0, c='b')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

elev = 10
azim = 65
ax.view_init(elev, azim)